# RAG para Video do Youtube


## Open Source

### Instalando e importando as dependências

In [1]:
%pip install sentence_transformers accelerate pytube langchain-google-genai chromadb langchain ctransformers faiss-gpu langchainhub --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 816.1/816.1 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 83.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from pytube import YouTube
from ctransformers import AutoModelForCausalLM
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

### Setup do modelo de transcrição Whisper

In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [4]:
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [5]:
model_id_whisper_v3 = "openai/whisper-large-v3"

model_whisper_v3 = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id_whisper_v3,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
    use_safetensors=True
)
model_whisper_v3.to(device)


processor_whisper_v3 = AutoProcessor.from_pretrained(model_id_whisper_v3)


pipe_whisper_v3 = pipeline(
    "automatic-speech-recognition",
    model=model_whisper_v3,
    tokenizer=processor_whisper_v3.tokenizer,
    feature_extractor=processor_whisper_v3.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Download Video do Youtube

In [9]:
# video_url = 'https://www.youtube.com/watch?v=jxYByG7gclA'
video_url = "https://www.youtube.com/watch?v=H_erG7HSK0A"

youtube = YouTube(video_url)

video_stream = youtube.streams.get_audio_only()

audio_file = video_stream.download()

print(f"Downloaded '{audio_file}'.")

Downloaded '/content/100 Food Hacks I Learned In Restaurants.mp4'.


### Usando Whisper para transcrição

In [10]:
# video_translation_whisper_v3 = pipe_whisper_v3(audio_file, generate_kwargs={"language": "portuguese", "task": "translate"})
video_translation_whisper_v3 = pipe_whisper_v3(audio_file)

In [11]:
print(video_translation_whisper_v3["text"])

 I spent my entire life learning what makes a great chef, but many of you know, I didn't go to culinary school I worked in restaurants in real-life scenarios learning real-life techniques and tricks now many of these tricks are industry secrets And you would never know unless you had done that before but the irony is that anyone could do them easily at home So today I'm going to show you a hundred of my favorite chef hacks that will ideally make your life easier make you a better Cook and maybe make you look like you know what you're doing. So let's begin. But before we get started with our chef hacks, I have something really important. The book tour is live. I'm coming to see you. There's gonna be books, there's gonna be hugging and kissing and talking and signing and free food. So here are the dates. The link is in the description if you wanna see more information. Come see How to properly wash produce. I'll be honest, I don't always do this, but when I remember to. Take three parts 

In [12]:
model_whisper_v3.cpu()
torch.cuda.empty_cache()

In [ ]:
with open('../content/drive/MyDrive/data-meetup-experience/video_translation.txt', 'w') as file:
    file.write(video_translation_whisper_v3["text"])

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

llama-2-7b-chat.Q2_K.gguf:   0%|          | 0.00/2.83G [00:00<?, ?B/s]

### Criando base de documentos para RAG

**Passo 1: Criar chunks do documento (transcrição do video):**





In [34]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=300,
#     chunk_overlap=60
# )


# loader = TextLoader(file_path='../content/drive/MyDrive/data-meetup-experience/video_translation.txt')
# docs = loader.load_and_split(text_splitter=text_splitter)

text_splitter = CharacterTextSplitter(
    separator=" ",
    chunk_size=300,
    chunk_overlap=60,
    length_function=len,
    is_separator_regex=False,
)

docs = text_splitter.create_documents([video_translation_whisper_v3['text']])


print(len(docs))
print(docs[1])

193
page_content="And you would never know unless you had done that before but the irony is that anyone could do them easily at home So today I'm going to show you a hundred of my favorite chef hacks that will ideally make your life easier make you a better Cook and maybe make you look like you know what you're"


**Passo 2: Carregando modelo de embedding Open Source**

In [35]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings


embedding_model_name = "BAAI/bge-small-en"

embedding_model_open_source = HuggingFaceBgeEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={"device": device},
    encode_kwargs={"normalize_embeddings": True}
)

**Passo 3: Transformando chunks em embeddings e salvando na base de conhecimento**

In [75]:
from langchain.vectorstores import FAISS

vector_store = FAISS.from_documents(docs, embedding_model_open_source)

Testando Retrieval

In [76]:
retriever = vector_store.as_retriever()

retriever.search_kwargs["fetch_k"] = 2
retriever.search_kwargs["maximal_marginal_relevance"] = True
retriever.search_kwargs["k"] = 2

In [77]:
question = "What is the best way to cut onions?"
context = retriever.get_relevant_documents(question)
context

[Document(page_content="When you're using raw onion on a dish eating it straight up I think in many cases is kind of crazy. I would highly recommend when you slice your onion after slicing instead of just putting it straight onto what you're gonna use it for, place it in a container, into the sink, cover with a wire rack,"),
 Document(page_content="you want. Stop your onions from making you cry. When you're cutting an onion, sure, not cutting into the core can sometimes help prevent crying, but for the most part, that's kind of bull . If your knife stole, you might start tearing up. If the onion gets torn somehow, you might start tearing up,")]

### Importando LLM Open Source

In [ ]:
from ctransformers import AutoModelForCausalLM

# check ctransformers doc for more configs
config = {'max_new_tokens': 128, 'temperature': 0}

llm = AutoModelForCausalLM.from_pretrained(
      "TheBloke/Llama-2-7B-chat-GGUF",
      model_type="llama",
      #lib='avx2', for cpu use
      gpu_layers=110, #110 for 7b, 130 for 13b
      **config
      )


### RAG

**Aplicando RAG**

In [46]:
# query = "What tips does the host give about onions?"
# query = "What tips are about tomatoes?"

query = "What is the best way to cut onions?"

context = retriever.get_relevant_documents(question)

prompt = f"""
Answer the question based on the provided context:
**Question:** {query}
**Context:** {context}
"""


In [47]:
response = llm(prompt, stream=False)

In [48]:
print(response)

The best way to cut onions is to use a mandolin or a sharp knife. These methods will help you achieve evenly sliced onions that are faster and safer than traditional cutting methods.


In [49]:
for doc in context:
  print(doc.page_content)

but the hardest to accept the secret to cooking evenly is even cuts now You could slice garlic or an onion or whatever you need to slice, or you could use a mandolin. And when you do this, not only is it faster and slightly more dangerous, but you end up with perfectly evenly sliced vegetables. This
When you're using raw onion on a dish eating it straight up I think in many cases is kind of crazy. I would highly recommend when you slice your onion after slicing instead of just putting it straight onto what you're gonna use it for, place it in a container, into the sink, cover with a wire rack,


## Gemini - Free!

**Passo 1: Criar chunks do documento (transcrição do video):**





In [78]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator=" ",
    chunk_size=300,
    chunk_overlap=60,
    length_function=len,
    is_separator_regex=False,
)

chunks = text_splitter.create_documents([video_translation_whisper_v3['text']])

print(len(chunks))
print(chunks[1])

193
page_content="And you would never know unless you had done that before but the irony is that anyone could do them easily at home So today I'm going to show you a hundred of my favorite chef hacks that will ideally make your life easier make you a better Cook and maybe make you look like you know what you're"


**Passo 2: Importando Embedding da Google**

In [52]:
from google.colab import userdata
import os

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')


**Passo 3: Transformando chunks em embeddings e salvando na base de conhecimento**

In [79]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI


embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")



vector_store_google = FAISS.from_documents(chunks, embeddings)


retriever_google = vector_store_google.as_retriever()

retriever_google.search_kwargs["fetch_k"] = 2
retriever_google.search_kwargs["maximal_marginal_relevance"] = True
retriever_google.search_kwargs["k"] = 2






In [80]:
question = "What is the best way to cut onions?"
context = retriever_google.get_relevant_documents(question)
context

[Document(page_content="you want. Stop your onions from making you cry. When you're cutting an onion, sure, not cutting into the core can sometimes help prevent crying, but for the most part, that's kind of bull . If your knife stole, you might start tearing up. If the onion gets torn somehow, you might start tearing up,"),
 Document(page_content="When you're using raw onion on a dish eating it straight up I think in many cases is kind of crazy. I would highly recommend when you slice your onion after slicing instead of just putting it straight onto what you're gonna use it for, place it in a container, into the sink, cover with a wire rack,")]

### RAG

**Aplicando RAG**

In [81]:
llm = ChatGoogleGenerativeAI(model="gemini-pro")


In [87]:
query = "What is the best way to cut onions?"

context = retriever_google.get_relevant_documents(question)



prompt = f"""
Answer the question based on the provided context:
**Question:** {query}
**Context:** {context}
"""

response = llm.invoke(prompt)


In [85]:
print(response)

content='The provided context does not contain information on how to cut onions. Therefore, I cannot answer the question.'


In [86]:
for doc in context:
  print(doc.page_content)

When you're using raw onion on a dish eating it straight up I think in many cases is kind of crazy. I would highly recommend when you slice your onion after slicing instead of just putting it straight onto what you're gonna use it for, place it in a container, into the sink, cover with a wire rack,
you want. Stop your onions from making you cry. When you're cutting an onion, sure, not cutting into the core can sometimes help prevent crying, but for the most part, that's kind of bull . If your knife stole, you might start tearing up. If the onion gets torn somehow, you might start tearing up,
